In [5]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import glob

In [6]:
root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/multitask-moe/COVID-CT-MD/"

In [7]:
df_annotation_2d = pd.read_csv(os.path.join(root, "index_2d.csv"))

In [8]:
df_annotation_2d

,clinical_data,Diagnosis,Folder,Patient Gender,Patient Age,Weight,Clinical characteristics,Surgery,Follow-up,PCR,Slice File,Slice Label
0,0,COVID-19,P001,M,039Y,80.0,"Dyspnea, Cough, Fever",NaN,NaN,NaN,IM0001,0.0
1,0,COVID-19,P001,M,039Y,80.0,"Dyspnea, Cough, Fever",NaN,NaN,NaN,IM0002,0.0
2,0,COVID-19,P001,M,039Y,80.0,"Dyspnea, Cough, Fever",NaN,NaN,NaN,IM0003,0.0
3,0,COVID-19,P001,M,039Y,80.0,"Dyspnea, Cough, Fever",NaN,NaN,NaN,IM0004,0.0
4,0,COVID-19,P001,M,039Y,80.0,"Dyspnea, Cough, Fever",NaN,NaN,NaN,IM0005,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
23344,306,Normal,normal076,M,030Y,72.0,"Chest pain, Cough, Loss of appetite",Yes,NaN,NaN,IM0167,0.0
23345,306,Normal,normal076,M,030Y,72.0,"Chest pain, Cough, Loss of appetite",Yes,NaN,NaN,IM0168,0.0
23346,306,Normal,normal076,M,030Y,72.0,"Chest pain, Cough, Loss of appetite",Yes,NaN,NaN,IM0169,0.0
23347,306,Normal,normal076,M,030Y,72.0,"Chest pain, Cough, Loss of appetite",Yes,NaN,NaN,IM0170,0.0


In [12]:
dict_out = {"id": [], "exam_id": [], "path": [], "class": [], "exam_class": []}

for i in range(len(df_annotation_2d)):
    item = df_annotation_2d.iloc[i]

    diagnosis = "Cap" if item["Diagnosis"] == "CAP" else item["Diagnosis"]
    folder = item["Folder"]

    slice_file = item["Slice File"]
    label = item["Slice Label"]

    path = f"./slices/{diagnosis} Cases/{folder}/{slice_file}.png"

    assert os.path.exists(os.path.join(root, path)), path

    if label == 1:
        if diagnosis == "COVID-19":
            cls = "atypical appearance pneumonia"
        elif diagnosis == "Cap":
            cls == "typical appearance pneumonia"
        else:
            raise NotImplementedError
    else:
        cls = "no findings"

    id = f"{diagnosis}/{folder}/{slice_file}"
    dict_out["id"].append(id)
    dict_out["exam_id"].append(id)
    dict_out["path"].append(path)

    dict_out["class"].append(cls)
    dict_out["exam_class"].append(cls)

df_out = pd.DataFrame.from_dict(dict_out)

In [13]:
df_out

,id,exam_id,path,class,exam_class
0,COVID-19/P001/IM0001,COVID-19/P001/IM0001,./slices/COVID-19 Cases/P001/IM0001.png,no findings,no findings
1,COVID-19/P001/IM0002,COVID-19/P001/IM0002,./slices/COVID-19 Cases/P001/IM0002.png,no findings,no findings
2,COVID-19/P001/IM0003,COVID-19/P001/IM0003,./slices/COVID-19 Cases/P001/IM0003.png,no findings,no findings
3,COVID-19/P001/IM0004,COVID-19/P001/IM0004,./slices/COVID-19 Cases/P001/IM0004.png,no findings,no findings
4,COVID-19/P001/IM0005,COVID-19/P001/IM0005,./slices/COVID-19 Cases/P001/IM0005.png,no findings,no findings
...,...,...,...,...,...
23344,Normal/normal076/IM0167,Normal/normal076/IM0167,./slices/Normal Cases/normal076/IM0167.png,no findings,no findings
23345,Normal/normal076/IM0168,Normal/normal076/IM0168,./slices/Normal Cases/normal076/IM0168.png,no findings,no findings
23346,Normal/normal076/IM0169,Normal/normal076/IM0169,./slices/Normal Cases/normal076/IM0169.png,no findings,no findings
23347,Normal/normal076/IM0170,Normal/normal076/IM0170,./slices/Normal Cases/normal076/IM0170.png,no findings,no findings


In [14]:
import numpy as np

df_out["dataset"] = "covid-ct-md"
df_out["class_num"] = 3
df_out["contains_negative"] = 1
df_out["modality"] = "ct"

df_out["split"] = "test"

In [15]:
df_out.to_csv(f"{root}/metadata.csv", index=False)

In [16]:
df_out["class"].value_counts()

class
no findings                      19534
atypical appearance pneumonia     3815
Name: count, dtype: int64